# Problem Formulation

## State Description

An environment dependent on it's own size where only one action is relevant: to clean out the grid so that every tile is 'off'. In order to do this, the program goes through every single tile and decides which tiles to turn according to a set pattern described in the transition model. 


Board size NxN
Cleaner 

## Inital State

Board(SIZE)

## Actions

* TAB

## Transition model

| Operator | Pre-conditions | Changes |
| -------- | -------------- | ------- |
| `tab(row, col)` | (row - 1, col) == true OR <br> (row, col - 1) == true OR <br> (row, col + 1) == true OR <br> (row + 1, col) == true | toggle value at (row - 1, col) AND <br> toggle value at (row, col - 1) AND <br> toggle value at (row, col + 1) AND <br> toggle value at (row + 1, col) |

## Goal State

```
sub allCellsAreEmpty(board)
	for row in board
		for cell in row
			if cell is true
				return false
	return true
```

## Path Cost

Because of the nature of the problem, the only measurable action is 'Tab'; this means that there will be a counter that will go up 1 point for everytime it turns a tile within the grid. There will be no counter for the amount of movements the program does when viewing the grid.

* TAB = +1

## Aima Dependencies

* search.py
* utils.py


In [40]:
from search import Problem
from search import astar_search

## Utilities

### Board

In [41]:
# converts a list of strings
# into a serialized version of a board
def toBoard(rows):
    return '\n'.join(rows)

# getAffectedCellsByPosition
# retrieves all affected cells based on the given position
# @param {Tuple} position
# @param {Integer} size
# @return {List<Tuple>}
def getAffectedCellsByPosition(position, size):
    row, col = position
    allAffectedCells = []
    # order matters
    if row - 1 >= 0: allAffectedCells.append((row - 1, col))
    if col - 1 >= 0: allAffectedCells.append((row, col - 1))
    if col + 1 < size: allAffectedCells.append((row, col + 1))
    if row + 1 < size: allAffectedCells.append((row + 1, col))

    return allAffectedCells

# buildGrid
# creates a string representation of a matrix based on the given size
# if filler is provided, tiles position is provided by the filler
# filler is meant to create specific matrix configurations
# @param {Integer} size (default: 5)
# @param {Function} filler (default: None)
# @return {String}
def buildGrid(size = 5, filler = None):
    rows = list()
    for r in range(size):
        row = list()
        for c in range(size):
            if filler:
                row.append(filler(r, c))
            else:
                row.append('0')
        rows.append('|'.join(row))
    return '\n'.join(rows)

# toggleCell
# toggles the cell's value based on a given location in the given encoded board
# @param {String} board
# @param {Tuple} position
# @return {String}
def toggleCell(board, position):
    rows = board.split('\n')
    affectedCells = getAffectedCellsByPosition(position, len(rows))
    for cell in affectedCells:
        row = rows[cell[0]].split('|')
        row[cell[1]] = '1' if row[cell[1]] == '0' else '0'
        rows[cell[0]] = '|'.join(row)
    return '\n'.join(rows)

# parseBoard
# parses the encoded board into a List<List<String>>
# @param {String} board
# @return {List<List<String>>}
def parseBoard(board):
    rows = []
    encodedRows = board.split('\n')
    for encodedRow in encodedRows:
        rows.append(encodedRow.split('|'))
    return rows


### Solution display

In [42]:
# displayProblemSolution
# prints the solution found by a search algorithm
# @param {Node} goal_node
# @param {String} methodName
# @param {Float} processingTime
def displayProblemSolution(goal_node, methodName, processingTime):
    print('METHOD: ' + methodName)
    print('--- search terminated after %s seconds ---' % (processingTime))

    if goal_node:
        actions = goal_node.solution()
        nodes = goal_node.path()
        print('SOLUTION')
        print('State:' )
        print(nodes[0].state)
        for na in range(len(actions)):
            print('ACTION: ' + str(actions[na]))
            print('State:')
            print(nodes[na+1].state)
            print('END \n')
    else:
        print('has not found a solution')

### Timer class

In [43]:
import time

# Timer computes the time taken by a process when executed
class Timer: 
    # inits Timer instance
    def __init__(self):
        self.timestamp = time.time()
        self.totalTime = 0
        
    # computes the difference between the current time and the timestamp
    def stop(self):
        self.totalTime = time.time() - self.timestamp
        
    # retrieves the difference between the current time and the timestamp
    def getTotalTime(self):
        return self.totalTime

### Problem runner

In [44]:
# run
# attempts to solve the given puzzle problem using the astar_search search and the given heuristic
# @param {CleanUpPuzzle} puzzle
# @param {String} heuristicType 
# @param {String} difficulty 
# @param {Function} h 
# @return {Float}
def run(puzzle = None, heuristicType = None, difficulty = None, h = None):
    watch = Timer()
    solution = astar_search(puzzle, h)
    watch.stop()
    if (PRINT_SOLUTION):
        displayProblemSolution(solution, 'astar_search (Heuristic type: ' + heuristicType + ', Difficulty ' + difficulty + ')', watch.getTotalTime())
    else: 
        print('astar_search (Heuristic type: ' + heuristicType + ', Difficulty ' + difficulty + ') = ' + str(watch.getTotalTime()))
    return watch.getTotalTime()

### Performance comparator

In [45]:
def comparePerformance(nonAdmissible, admissible):
    indexes = ['EASY', 'MEDIUM', 'HARD']
    cols = ['Difficulty', 'Non Admissible Time (NA)', 'Admissible Time (A)', 'Time Diff (A - NA)', 'NA Time Saving %']
    headers = []
    rows = []
    INDEX_TEMPLATE = ' {:<10}'
    STR_TEMPLATE = ' {:<25}'
    NUM_TEMPLATE = ' {:<25.10f}'
    PER_TEMPLATE = ' {:<25}'
    
    for col in cols:
        if col == 'Difficulty':
            headers.append(INDEX_TEMPLATE.format(col))
        else:
            headers.append(STR_TEMPLATE.format(col))
    rows.append('|'.join(headers))
    
    for index in indexes:
        nonAdmissibleTime = nonAdmissible[index]
        admissibleTime = admissible[index]
        difference = admissible[index] - nonAdmissible[index]
        performance = (difference / nonAdmissibleTime) * 100
        label = ' slower'
        if performance > 0:
            label = ' faster'
        rows.append(
            '|'.join([
                INDEX_TEMPLATE.format(index),
                NUM_TEMPLATE.format(nonAdmissibleTime),
                NUM_TEMPLATE.format(admissibleTime),
                NUM_TEMPLATE.format(difference),
                PER_TEMPLATE.format('{:.5f}'.format(abs(performance)) + label)
            ])
        )
    print('\n'.join(rows))

### Boarding Configurations And Global Variables

In [46]:
# grid generator https://jsbin.com/hibuyuzemo/edit?html,js,console,output
EASY_BOARD = [
    '0|0|0|0|0|0|0|0|0|0|0',
    '0|0|0|0|0|0|0|0|0|0|0',
    '0|0|0|0|1|0|0|0|0|0|0',
    '0|0|0|1|0|1|0|0|0|0|0',
    '0|0|0|0|1|0|0|0|0|0|0',
    '0|0|0|0|0|0|0|0|0|0|0',
    '0|1|0|1|0|0|0|0|0|0|0',
    '1|0|0|0|1|0|0|1|0|0|0',
    '0|1|0|1|0|0|1|0|1|0|0',
    '0|0|0|0|0|1|0|1|0|0|0',
    '0|0|0|0|1|0|1|0|0|0|0'
]


MEDIUM_BOARD = [
    '0|0|0|1|0|0|0|0|0|0|0',
    '0|0|1|0|1|0|0|0|0|0|0',
    '0|0|0|1|0|0|0|0|0|0|0',
    '0|0|0|0|0|0|0|1|0|0|0',
    '0|0|0|1|0|0|1|0|0|0|0',
    '0|0|1|0|1|0|0|0|0|1|0',
    '0|0|0|1|0|0|0|0|1|1|0',
    '0|0|0|0|0|0|0|0|1|0|1',
    '1|0|0|0|0|1|0|0|0|1|0',
    '0|1|0|0|1|0|1|0|0|0|0',
    '1|0|0|0|0|1|0|0|0|0|0'
]

HARD_BOARD = [
    '0|0|0|0|0|0|0|1|0|0|0',
    '0|0|1|0|1|0|0|1|1|0|0',
    '0|0|0|1|0|1|1|1|1|0|0',
    '1|0|0|0|0|0|0|1|0|0|0',
    '0|1|0|0|0|0|0|0|0|0|0',
    '1|0|0|0|0|0|0|0|0|0|1',
    '0|0|0|0|0|0|0|0|0|1|0',
    '0|0|0|0|0|0|0|0|1|1|1',
    '0|0|0|0|1|0|0|1|1|1|1',
    '1|0|0|1|0|1|0|0|1|0|0',
    '0|1|0|0|1|0|0|0|1|0|1'
]

BOARD_SIZE = 11
PRINT_SOLUTION = True

## Clean Up Puzzle Problem Implementation

In [47]:
#### allow actions
TAB = 'TAB'

class CleanUpPuzzle(Problem):
    def __init__(self, initial, goal):
        Problem.__init__(self, initial, goal)
       
    # actions
    # computes all possible movements
    # that can be perfomed on the state
    def actions(self, state):
        return getPossibleMovements(state)

    # result
    # changes the state based on a given action
    def result(self, state, action):
        return toggleCell(state, action[1])

# getPossibleMovements scans the full board
# to retrieve possible movements
# @param {String} state
# @return {List<Tuple>}
def getPossibleMovements(state):
    board = parseBoard(state)
    size = len(board)
    actions = []
    for row in range(size):
        for col in range(size):
            affectedCells = getAffectedCellsByPosition((row, col), size)
            for cell in affectedCells:
                if board[cell[0]][cell[1]] == '1':
                    actions.append((TAB, (row, col)))
                    break
    return actions

## Heuristics

### Non Admissible

this computes the percentage of cells occupied in the board at a given state (Node)

In [48]:
# nonAdmissibleHeuristic
# this computes the percentage of cells occupied in the board at a given state (Node)
def nonAdmissibleHeuristic(node):
    size = len(node.state.split('\n'))
    remaningTilesCount = node.state.count('1')
    return (remaningTilesCount / (size*size)) * 100

### Admissible
 
After creating a minimalistic version of the game ([it can be found here](https://jsbin.com/hibuyuzemo/edit?html,js,output)) and playing for some time, we notice the following pattern:

* Clearing all tiles from row 0 (By clicking the cell located below the one you want to remove)
* Clearing all tiles from row 1 (By clicking the cell located below the one you want to remove)
* and So on
* Clearing all tiles from row SIZE - 2 (By clicking the cell located below the one you want to remove in this case clicking cell in the last row)

This actions always lead to the last row having no tiles to remove (which is the goal state). So counting the number of tiles from the row at the index 0 to row at index SIZE - 2 (Where SIZE is the Board SIZE) is sufficient to comply with the admissible heuristic definition that states the following `0 <= h(n) <= h*(n)`.

> Caveat: This works only for solvable board configurations


In [49]:
# admissibleHeuristic
# this counts the number of tiles from the row at the index 1 to row at index SIZE - 1 (Where SIZE is the Board SIZE).
def admissibleHeuristic(node):
    rows = node.state.split('\n')
    count = 0
    # last row is not considered because if the previous rows do not have tiles
    # by default the last row will not have tiles (This works only for solvable board configurations)
    for index in range(len(rows) - 1):
        count = count + rows[index].count('1')
    return count

## Heuristic Execution

In [50]:
GOAL_STATE = buildGrid(BOARD_SIZE)
nonAdmissibleResultsByDifficulty = dict()

# Non Admissible Heuristic
nonAdmissibleResultsByDifficulty['EASY'] = run(CleanUpPuzzle(toBoard(EASY_BOARD), GOAL_STATE), 'Non admissible heuristic', 'EASY', nonAdmissibleHeuristic)
nonAdmissibleResultsByDifficulty['MEDIUM'] = run(CleanUpPuzzle(toBoard(MEDIUM_BOARD), GOAL_STATE), 'Non admissible heuristic', 'MEDIUM', nonAdmissibleHeuristic)
nonAdmissibleResultsByDifficulty['HARD'] = run(CleanUpPuzzle(toBoard(HARD_BOARD), GOAL_STATE), 'Non admissible heuristic','HARD', nonAdmissibleHeuristic)

METHOD: astar_search (Heuristic type: Non admissible heuristic, Difficulty EASY)
--- search terminated after 0.0028069019317626953 seconds ---
SOLUTION
State:
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|1|0|0|0|0|0|0
0|0|0|1|0|1|0|0|0|0|0
0|0|0|0|1|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
1|0|0|0|1|0|0|1|0|0|0
0|1|0|1|0|0|1|0|1|0|0
0|0|0|0|0|1|0|1|0|0|0
0|0|0|0|1|0|1|0|0|0|0
ACTION: ('TAB', (3, 4))
State:
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
1|0|0|0|1|0|0|1|0|0|0
0|1|0|1|0|0|1|0|1|0|0
0|0|0|0|0|1|0|1|0|0|0
0|0|0|0|1|0|1|0|0|0|0
END 

ACTION: ('TAB', (8, 7))
State:
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
1|0|0|0|1|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
0|0|0|0|0|1|0|0|0|0|0
0|0|0|0|1|0|1|0|0|0|0
END 

ACTION: ('TAB', (10, 5))
State:
0|0|0|0|0

In [51]:
admissibleResultsByDifficulty = dict()
# Admissible Heuristic
admissibleResultsByDifficulty['EASY'] = run(CleanUpPuzzle(toBoard(EASY_BOARD), GOAL_STATE), 'Admissible', 'EASY', admissibleHeuristic)
admissibleResultsByDifficulty['MEDIUM'] = run(CleanUpPuzzle(toBoard(MEDIUM_BOARD), GOAL_STATE), 'Admissible', 'MEDIUM', admissibleHeuristic)
admissibleResultsByDifficulty['HARD'] = run(CleanUpPuzzle(toBoard(HARD_BOARD), GOAL_STATE), 'Admissible','HARD', admissibleHeuristic)

METHOD: astar_search (Heuristic type: Admissible, Difficulty EASY)
--- search terminated after 0.0024640560150146484 seconds ---
SOLUTION
State:
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|1|0|0|0|0|0|0
0|0|0|1|0|1|0|0|0|0|0
0|0|0|0|1|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
1|0|0|0|1|0|0|1|0|0|0
0|1|0|1|0|0|1|0|1|0|0
0|0|0|0|0|1|0|1|0|0|0
0|0|0|0|1|0|1|0|0|0|0
ACTION: ('TAB', (3, 4))
State:
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
1|0|0|0|1|0|0|1|0|0|0
0|1|0|1|0|0|1|0|1|0|0
0|0|0|0|0|1|0|1|0|0|0
0|0|0|0|1|0|1|0|0|0|0
END 

ACTION: ('TAB', (8, 7))
State:
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
1|0|0|0|1|0|0|0|0|0|0
0|1|0|1|0|0|0|0|0|0|0
0|0|0|0|0|1|0|0|0|0|0
0|0|0|0|1|0|1|0|0|0|0
END 

ACTION: ('TAB', (7, 1))
State:
0|0|0|0|0|0|0|0|0|0|0
0|

## Non Admissible vs  Admissible

In [52]:
comparePerformance(nonAdmissibleResultsByDifficulty, admissibleResultsByDifficulty)

 Difficulty| Non Admissible Time (NA) | Admissible Time (A)      | Time Diff (A - NA)       | NA Time Saving %         
 EASY      | 0.0028069019             | 0.0024640560             | -0.0003428459            | 12.21439 slower          
 MEDIUM    | 0.0060667992             | 0.0076780319             | 0.0016112328             | 26.55820 faster          
 HARD      | 0.7200970650             | 4.9939227104             | 4.2738256454             | 593.50688 faster         


## Conclusions

#### Could they solve all or some problems?

Yes, Both heuristics were able to solve the problems in a fairly reasonable time.

#### How efficient were they?

Both heuristics were efficient at solving the easy and medium complex board. Regarding the complex one, both showed an increase in their respective execution time time but overall the non-admissible heuristic peformed better than the admissible one (this might be because our heuristic is probably not the best one).

#### Was it difficult to program the heuristics?

No, it was not. The implementations are small and simple (at least for the solutions provided in this assignment).

#### What was the most difficult?

The most difficult aspect was to find a pattern that could be used as an admissible heuristic (a pattern that complies with this formula `0 <= h(n) <= h*(n)`) and prove that it is indeed true for _all the cases_. 

#### which heuristic seems to be the best for the selected problems? 

Based on the execution and after running a comparison between an admissible and non-admissible heuristic, the non-admissible heuristic seems to be the best solution for this puzzle based on how well it performed when tested with complex board configuration (its execution time was, sometimes, less than 1 second and the admissible heuristic execution time takes at least 4 seconds). However, this big gap was only consistent when comparing the execution time of both heuristics for the most complex board configuration (when they are tested using an easy and medium complex board the time difference was not that big).
